In [14]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [15]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [17]:
df_green.registerTempTable('green')

In [18]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1,2
""")

In [19]:
df_green_revenue.show(20)

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 00:00:00|   7| 769.7299999999997|            45|
|2020-01-01 00:00:00|  17|            195.03|             9|
|2020-01-01 00:00:00|  18|               7.8|             1|
|2020-01-01 00:00:00|  22|              15.8|             1|
|2020-01-01 00:00:00|  24|              87.6|             3|
|2020-01-01 00:00:00|  25|             531.0|            26|
|2020-01-01 00:00:00|  29|              61.3|             1|
|2020-01-01 00:00:00|  32| 68.94999999999999|             2|
|2020-01-01 00:00:00|  33|317.27000000000004|            11|
|2020-01-01 00:00:00|  35|129.95999999999998|             5|
|2020-01-01 00:00:00|  36|295.34000000000003|            11|
|2020-01-01 00:00:00|  37|175.67000000000002|             6|
|2020-01-01 00:00:00|  38| 98.78999999999999|             2|
|2020-01-01 00:00:00|  4

In [20]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

24/03/06 17:39:03 WARN MemoryManager: Total allocation exceeds 95.00% (1’020’054’720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [21]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [22]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [23]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

24/03/06 17:39:41 WARN MemoryManager: Total allocation exceeds 95.00% (1’020’054’720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/06 17:39:42 WARN MemoryManager: Total allocation exceeds 95.00% (1’020’054’720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [24]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [25]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [26]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [27]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

24/03/06 18:03:09 WARN MemoryManager: Total allocation exceeds 95.00% (1’020’054’720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [28]:
df_join = spark.read.parquet('data/report/revenue/total')

In [29]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [30]:
df_zones = spark.read.parquet('zones/')

In [31]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [32]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')

24/03/06 18:15:58 WARN MemoryManager: Total allocation exceeds 95.00% (1’020’054’720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [33]:
df_result.show(20)

+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|LocationID|  Borough|                Zone|service_zone|
+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|2020-01-01 00:00:00|   4|              NULL|                NULL|1004.3000000000002|                   57|         4|Manhattan|       Alphabet City| Yellow Zone|
|2020-01-01 00:00:00|  15|              NULL|                NULL|             34.09|                    1|        15|   Queens|Bay Terrace/Fort ...|   Boro Zone|
|2020-01-01 00:00:00|  35|129.95999999999998|                   5|              NULL|                 NULL|        35| Brooklyn|         Brownsville|   Boro Zone|
|2020-01-01 00:00:00| 

24/03/06 18:59:25 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 901619 ms exceeds timeout 120000 ms
24/03/06 18:59:25 WARN SparkContext: Killing executors is not supported by current scheduler.
24/03/06 18:59:34 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$